In [46]:
pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
import re

from __future__ import unicode_literals
from hazm import *

In [48]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

path = "/content/drive/My Drive/Snappfood - Sentiment Analysis.csv"
df = pd.read_csv(path,sep='\t')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


#a)

In [49]:
df.isnull().sum()

Unnamed: 0    69480
comment           0
label             0
label_id        520
dtype: int64

In [50]:
df.drop(columns=['Unnamed: 0','label'] , inplace=True)
df.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0


In [51]:

df.info()
df.isnull().sum()
df.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   comment   70000 non-null  object 
 1   label_id  69480 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


In [52]:
df['label_id']=df['label_id'].astype(int)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69480 entries, 0 to 69999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comment   69480 non-null  object
 1   label_id  69480 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.6+ MB


,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [53]:
def remove_tags(string):
    result = re.sub('','',string)          #remove HTML tags
    result = re.sub('https://.*','',result)   #remove URLs
    return result

In [54]:
df['comment']=df['comment'].apply(lambda cw : remove_tags(cw)) 
df.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [55]:
import codecs

path = "/content/drive/My Drive/persian"
with open(path,encoding="utf8") as stop_word_file:
    stop_words=stop_word_file.readlines()
    
stop_words = [line.replace('\n' , '') for line in stop_words]

In [56]:
max_len = 0
for index , row in df.iterrows():
    sentence_list=[]
    comment = row['comment']
    com_len = len(comment.split(' '))
    if com_len>max_len:
        max_len = com_len
    for word in comment.split():
        if word not in stop_words:
            sentence_list.append(word)
            
    df.loc[index]={
        'comment':' '.join(sentence_list),
        'label_id':row['label_id']
    }

In [57]:
df.head()

,comment,label_id
0,حیف وقت بنویسم سرویس دهیتون افتضاح,1
1,قرار ساعته برسه نیم ساعت موقع رسید، ببین چقدرر...,0
2,قیمت مدل کیفیتش سازگاری نداره، ظاهر فریبنده دا...,1
3,عالللی اندازه کیفیت خوب، امیداورم کیفیتتون باش...,0
4,شیرینی وانیلی مدل بود.,0


In [58]:
lemmatizer = Lemmatizer()
normalizer = Normalizer()
def lemmatize_text(text):
    st = ""
    res = normalizer.normalize(text)
    for w in word_tokenize(res):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['comment'] = df.comment.apply(lemmatize_text)

In [59]:
df.head()

,comment,label_id
0,حیف وقت نوشت#نویس سرویس دهیتون افتضاح,1
1,قرار ساعته برسه نیم ساعت موقع رسید ، دید#بین چ...,0
2,قیمت مدل کیفیت سازگار نداره ، ظاهر فریبنده دار...,1
3,عالللی اندازه کیفیت خوب ، امیداورم کیفیتتون با...,0
4,شیرینی وانیل مدل بود#باش .,0


In [60]:
comment = df['comment'].values
label_id = df['label_id'].values

In [61]:
train_comment, test_comment, train_labels, test_labels = train_test_split(comment, label_id, stratify = label_id, test_size=0.2, random_state=42)

#b)

In [62]:
word_counts = {}

In [63]:
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_comment)
vocab = vec.get_feature_names_out()
X = X.toarray()

for l in range(2):
    word_counts[l] = defaultdict(lambda: 0)
for i in range(X.shape[0]):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]

In [64]:
def fit(x, y, labels):

    n_label_items = {}
    log_label_priors = {}
    n = len(x)

    grouped_data = {}
    for l in labels:
        grouped_data[l] = x[np.where(y == l)]

    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors

In [65]:
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(word_tokenize(text))
        for word in words:
            if word not in vocab: continue
            for l in labels:
              
                # calculate proability 
                a = word_counts[l][word] + 1
                b = n_label_items[l] + len(vocab)
                log_w_given_l =  math.log(a/b)
                label_scores[l] += log_w_given_l

        result.append(max(label_scores, key=label_scores.get))
    return result

#c)

In [66]:
labels = [0,1]
n_label_items, log_label_priors = fit(train_comment,train_labels,labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_comment)

In [67]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = accuracy_score(test_labels,pred)
precision = precision_score(test_labels,pred)
recall = recall_score(test_labels,pred)
f1_score = f1_score(test_labels,pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1_score}")

Accuracy: 0.7784254461715602
Precision: 0.7024714828897338
Recall: 0.9621003905684942
F1 score: 0.8120383370978572
